## Dram Data Upload

In this assignment, you'll upload Dram shop data to your GBQ account and run a couple of queries against it. The data for this assignment is in a .csv file callled `items-2022-01-01-2022-10-01.csv`. Save this file into the same folder as this repository. This is an example of item-level reports exported from the point-of-sale system at the Dram shop.

Your goals are to upload this file to a table in a GBQ data set that you create. Call the table `dram_items_2022`. 
As always, make sure _not_ to commit any big data files or API keys to your repo.

Make your data pipeline idempotent, which means you will be checking for the presence of your tables and, if they exist, deleting them before recreating them. 

In this assignment you'll use the `pyjanitor` package, which you can install with `pip install pyjanitor`. Then the `import janitor` call will run. This package, which is modeled on the `janitor` package in R, can be used to clean up names in a data frame. If you have a Pandas data frame called `df`, then `df = janitor.clean_names(df)` will give you nice lowercase names with underscores. You can learn more [here](https://github.com/pyjanitor-devs/pyjanitor). 


In [ ]:
import os
import re
import datetime 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_gbq
import janitor

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

### GBQ Set Up

In this next section we connect to our GBQ project and list the data sets inside to test the connection.

In [ ]:
# These first two values will be different on your machine. 
service_path = "/Users/chandler/Dropbox/Teaching/"
service_file = 'umt-msba-037daf11ee16.json' # change this to your authentication information  
gbq_proj_id = 'umt-msba' # change this to your project. 

# And this should stay the same. 
private_key =service_path + service_file

In [ ]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [ ]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [ ]:
for item in client.list_datasets() : 
    print(item.full_dataset_id)

### Checking for and deleting monthly tables

In this section, check for a table with the name `dram_items_2022`. If that table exists, delete it with the `delete_table` function. 

In [ ]:
# Your code here





### Reading in and modifying data

In this section, read in the data using the Pandas `read_csv` function. Once you've read in the data, make the following modifications to it: 

* Convert the fields that have dollar signs (such as `gross_sales`) into numeric data. Watch out for dollar signs and commas.
* Change the type of the column `modifiers_applied` to string.
* Replace the `sku` column with a column of empty strings. 
* Clean the names with the janitor package.


In [ ]:
# Your code here.







### Upoad your data to GBQ

Using the Pandas function `to_gbq`, upload your data to GBQ.

In [ ]:
# Your code here





### Query your data

Let's start by counting the rows in your data. Here's an example of how to do that, though you'll need to update the project and data set parameters in the query below. Note that the query inside the triple quotes can just be run the console. 

In [ ]:
# Your code here 

query = """
    SELECT COUNT(*)
    FROM `umt-msba.dram_shop.dram_items_2022`
"""

results = client.query(query)


for row in results :
    print(f'There were {row[0]} rows.')


You can also bring results back into a data frame with `pandas_gbq`. 

In [ ]:
# Your code here 

query = """
    SELECT customer_name, customer_id, SUM(gross_sales) AS gross_sales
    FROM `umt-msba.dram_shop.dram_items_2022`
    WHERE customer_name != ""
    GROUP BY customer_name, customer_id
    ORDER BY gross_sales DESC
"""

customer_spend = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)

customer_spend.head()

In [ ]:
customer_spend.describe()

Now some additional work for you. Write a query that returns sales by day by category for the Front Street location to a data frame. Call this data frame `daily_category_sales` with columns `date`, `category`, and `gross_sales` (which is the sum of sales for that date and category.

If you've done this correctly, the cells below should run correctly and summarize the data for you and do some plotting. 

In [ ]:
# Your code here 





In [ ]:
category_summary = daily_category_sales.groupby('category').sum('gross_sales').nlargest(10,'gross_sales').reset_index()
category_summary

In [ ]:
for category in category_summary.category : 
    
    df = daily_category_sales.query(f"category == '{category}'")
    df = df.sort_values('date', ascending=True)
    plt.plot(df['date'], df['gross_sales'])
    plt.xticks(ticks=range(1,300,30),rotation='vertical')
    plt.title(category)
    plt.show()

